## Subways data ETL process

Installing packages

In [ ]:
!pip install xlsxwriter

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 149 kB 5.1 MB/s 


#### Mounting gdrive storage

In [ ]:
#mounts your personal drive on this notebook
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#### Importing libraries

In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import json
import numpy as np
import xlsxwriter
from datetime import datetime

#### Logging process to logfile
When creating an ETL pipeline, we should always think of it as an automated process, then it is always useful to log as much information as possible to make the debugging later if necessary, in case there occured a problem, easier.
In this project, I create a text file and log the start and end parts of each pipeline steps. In a real ETL pipeline, one could also log information about catched errors or error messages in case there are some.

#### Defining functions

In [ ]:
def log(message):
    # set timestamp format
    timestamp_format = '%Y-%h-%d-%H:%M:%S'
    # get current timestamp
    now = datetime.now()
    timestamp = now.strftime(timestamp_format)
    with open("/content/drive/MyDrive/analysis/logs/logfile.txt","a") as f:
        f.write(timestamp + ',' + message + '\n')

def parse_tables(tables,idx):
    # storing the target table
    table = tables[idx]
    # converting table html code to pandas df
    data = pd.read_html(str(table))
    # storing the parsed data in a dataframe
    df = pd.DataFrame(data[0]) 
    return df  

## Extract

he Extract part is the first step of every pipeline you intend to do, so the data in this one is mainly extracted from Wikipedia in the following [link](https://en.wikipedia.org/wiki/List_of_metro_systems)

In [ ]:
# create log message
log("Start Extract Part")

### Webscraping the list of subway systems
Let's now scrape the Wikipedia page containing the data that we need to manipulate through the ETL process.

In [ ]:
# create log message
log("Start Webscraping Wikipedia Subways Data")

In [ ]:
# get html data first
html_data = requests.get("https://en.wikipedia.org/wiki/List_of_metro_systems")

# check if status is 200 -> shows that its allowed to scrape the webpage
try:
    print(f'HTML request to website was successful with code: {html_data.status_code}')
except:
    print(f'HTML request was not successful, code: {html_data.status_code}')  

HTML request to website was successful with code: 200


#### Parsing data from HTML table

In [ ]:
# parse html data now using BeautifulSoup
soup = BeautifulSoup(html_data.text, "html.parser")
# get all tables from wikipedia page
tables = soup.find_all('table',{'class':"wikitable"})

#### Visualizing data of the main table inside the tables object

In [ ]:
## Storing the data of the first HTML table into a dataframe by using the parse_tables function
df_list = parse_tables(tables,0)
df_list.head()

,City,Country/region,Name,Year .mw-parser-output .tooltip-dotted{border-bottom:1px dotted;cursor:help}opened,Year of lastexpansion,Stations,System length,Annual ridership(millions)
0,Algiers,Algeria,Algiers Metro,2011[12],2018[13],19[13],18.5 km (11.5 mi)[14],45.3 (2019)[R 1]
1,Buenos Aires,Argentina,Buenos Aires Underground,1913[Nb 1],2019[17],90[18],56.7 km (35.2 mi)[18],74.0 (2020)[R 2]
2,Yerevan,Armenia,Yerevan Metro,1981[19],1996[20],10[19],13.4 km (8.3 mi)[19],10.75 (2020)[R 3]
3,Sydney,Australia,Sydney Metro,2019[21],–,13[21],36 km (22 mi)[21][22],12.9 (2020)[R 4][R Nb 1]
4,Vienna,Austria,Vienna U-Bahn,1976[23][Nb 2],2017[24],98[25],83.3 km (51.8 mi)[23],459.8 (2019)[R 6]


In [ ]:
## Storing the data of the second HTML table into a dataframe by using the parse_tables function
df_by_country = parse_tables(tables,1)
df_by_country.head()

,Country,Systems,Length,Lines,Stations,Annual ridership / km (millions),Inauguration
0,Poland,1,35.5 km (22.1 mi),2,34,195.4 (2019),1995
1,China,44,"8,684.95 km (5,396.58 mi)",250,5386,2.10 (2020)[R Nb 26],1969
2,United States,15,"1,344.7 km (835.6 mi)",86,1228,0.75 (2020),1892
3,South Korea,8,871.2 km (541.3 mi),34,728,3.87 (2019),1974
4,Japan,13,791.2 km (491.6 mi),45,747,8.19 (2019),1927


In [ ]:
## Storing the data of the third HTML table into a dataframe by using the parse_tables function
df_under_const = parse_tables(tables,2)
df_under_const.head()

,City,Country,Name,Start of construction,Planned opening
0,Dhaka,Bangladesh,Dhaka Metro,2016[UC 2],2022[UC 3]
1,Guilin,China,Guilin Metro,2017[UC 4],2025
2,Liuzhou,China,Liuzhou Metro,2016,2024
3,Nantong,China,Nantong Metro,2017,2022
4,Qingyuan,China,Qingyuan Maglev,2019,2022


In [ ]:
df_list.columns

Index(['City', 'Country/region', 'Name',
       'Year .mw-parser-output .tooltip-dotted{border-bottom:1px dotted;cursor:help}opened',
       'Year of lastexpansion', 'Stations', 'System length',
       'Annual ridership(millions)'],
      dtype='object')

## Transform

In [ ]:
## Renaming columns and rearranging order of columns
df_list.rename(columns = {'Country/region':'Country',
                          'Name':'System_Name',
                          'Year .mw-parser-output .tooltip-dotted{border-bottom:1px dotted;cursor:help}opened':'Year_Opened',
                          'Year of lastexpansion':'Year_of_Last_Expansion',
                          'System length':'System_length',
                          'Annual ridership(millions)':'Annual_Ridership'}, inplace = True)

#Keeping just numbers on columns
df_list.Year_Opened = df_list.Year_Opened.str.extract('(\d+)')
df_list.Year_of_Last_Expansion = df_list.Year_of_Last_Expansion.str.extract('(\d+)')
df_list.System_length = df_list.System_length.str.extract('(\d+)')
df_list.Stations = df_list.Stations.str.extract('(\d+)')
df_list.Annual_Ridership = df_list.Annual_Ridership.str.extract('(\d+)')

df_list = df_list[['Country', 'City', 'System_Name', 'Year_Opened', 'Year_of_Last_Expansion',
                   'Stations','System_length','Annual_Ridership']]
df_list.head()

,Country,City,System_Name,Year_Opened,Year_of_Last_Expansion,Stations,System_length,Annual_Ridership
0,Algeria,Algiers,Algiers Metro,2011,2018,19,18,45
1,Argentina,Buenos Aires,Buenos Aires Underground,1913,2019,90,56,74
2,Armenia,Yerevan,Yerevan Metro,1981,1996,10,13,10
3,Australia,Sydney,Sydney Metro,2019,NaN,13,36,12
4,Austria,Vienna,Vienna U-Bahn,1976,2017,98,83,459


In [ ]:
#Checking columns data types
df_list.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 201 entries, 0 to 200
Data columns (total 8 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Country                 201 non-null    object
 1   City                    201 non-null    object
 2   System_Name             201 non-null    object
 3   Year_Opened             201 non-null    object
 4   Year_of_Last_Expansion  181 non-null    object
 5   Stations                201 non-null    object
 6   System_length           201 non-null    object
 7   Annual_Ridership        188 non-null    object
dtypes: object(8)
memory usage: 12.7+ KB


In [ ]:
#Filling NA values
df_list.fillna(0)

,Country,City,System_Name,Year_Opened,Year_of_Last_Expansion,Stations,System_length,Annual_Ridership
0,Algeria,Algiers,Algiers Metro,2011,2018,19,18,45
1,Argentina,Buenos Aires,Buenos Aires Underground,1913,2019,90,56,74
2,Armenia,Yerevan,Yerevan Metro,1981,1996,10,13,10
3,Australia,Sydney,Sydney Metro,2019,0,13,36,12
4,Austria,Vienna,Vienna U-Bahn,1976,2017,98,83,459
...,...,...,...,...,...,...,...,...
196,United States,San Juan,Tren Urbano,2004,2005,16,17,1
197,United States,"Washington, D.C.",Washington Metro,1976,2014,91,188,57
198,Uzbekistan,Tashkent,Tashkent Metro,1977,2020,39,57,38
199,Venezuela,Caracas,Caracas Metro[Nb 81],1983,2015,52,67,358


In [ ]:
#Converting data types
df_list.Year_Opened = df_list.Year_Opened.astype(int)
df_list.Year_of_Last_Expansion = pd.to_numeric(df_list.Year_of_Last_Expansion, errors='coerce')
df_list.Stations = df_list.Stations.astype(int)
df_list.System_length = df_list.System_length.astype(int)
df_list.Annual_Ridership = pd.to_numeric(df_list.Annual_Ridership, errors='coerce')
#Checking DataTypes
df_list.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 201 entries, 0 to 200
Data columns (total 8 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Country                 201 non-null    object 
 1   City                    201 non-null    object 
 2   System_Name             201 non-null    object 
 3   Year_Opened             201 non-null    int64  
 4   Year_of_Last_Expansion  181 non-null    float64
 5   Stations                201 non-null    int64  
 6   System_length           201 non-null    int64  
 7   Annual_Ridership        188 non-null    float64
dtypes: float64(2), int64(3), object(3)
memory usage: 12.7+ KB


In [ ]:
df_list.head()

,Country,City,System_Name,Year_Opened,Year_of_Last_Expansion,Stations,System_length,Annual_Ridership
0,Algeria,Algiers,Algiers Metro,2011,2018.0,19,18,45.0
1,Argentina,Buenos Aires,Buenos Aires Underground,1913,2019.0,90,56,74.0
2,Armenia,Yerevan,Yerevan Metro,1981,1996.0,10,13,10.0
3,Australia,Sydney,Sydney Metro,2019,NaN,13,36,12.0
4,Austria,Vienna,Vienna U-Bahn,1976,2017.0,98,83,459.0


## Loading

In [ ]:
#Sending the first dataframe of data to Gdrive storage as an .xlsx file
df_list.to_excel('/content/drive/MyDrive/analysis/data/subways_list',
                 engine='xlsxwriter',
                 index=False)

In [ ]:
#Sending the second dataframe of data to Gdrive storage as an .xlsx file
df_by_country.to_excel('/content/drive/MyDrive/analysis/data/subways_list_by_country',
                        engine='xlsxwriter',
                        index=False)

In [ ]:
#Sending the second dataframe of data to Gdrive storage as an .xlsx file
df_under_const.to_excel('/content/drive/MyDrive/analysis/data/subways_list_under_const',
                        engine='xlsxwriter',
                        index=False)

In [ ]:
#Storing data in csv files
df_list.to_csv('/content/drive/MyDrive/analysis/data/subways_list.csv',index=False)
df_by_country.to_csv('/content/drive/MyDrive/analysis/data/subways_list_by_country.csv',index=False)
df_under_const.to_csv('/content/drive/MyDrive/analysis/data/subways_list_under_const.csv',index=False)